In [18]:
#Adds one second and converts to statsbomb - test file as a backup
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join
import re
game_dir = "/home/lz80/rdf/sp161/shared/soccer-decision-making/HawkEye/HawkeyeUnzipped/"
games = pd.read_json("/home/lz80/rdf/sp161/shared/soccer-decision-making/womens_euro_receipts/matches/53/106.json", convert_dates = False)
games['home_team'] = games.apply(lambda d: d['home_team']['home_team_name'], axis = 1).str.replace("Women's", "").str.replace("WNT", "").str.strip()
games['away_team'] = games.apply(lambda d: d['away_team']['away_team_name'], axis = 1).str.replace("Women's", "").str.replace("WNT", "").str.strip()
#dirfiles = [f for f in listdir(dir_path) if isfile(join(dir_path, f))]
dirfiles = [f for f in listdir(game_dir) if not f.startswith('.')]
matches_map = {}
for game in dirfiles:
    home_team = game.split('_')[1]
    away_team = game.split('_')[2]
    #print(games['away_team'])
    game_id = games[(games['home_team'] == home_team) & (games['away_team'] == away_team)].reset_index().loc[0]['match_id']
    matches_map[game] = game_id

In [19]:
matches_map

{'2032235_Germany_France': 3845507,
 '2032215_Germany_Spain': 3835330,
 '2032216_Finland_Germany': 3835338,
 '2032231_Germany_Austria': 3844385,
 '2032213_Germany_Denmark': 3835322,
 '2032236_England_Germany': 3847567}

In [20]:
def convert_pos(coords, needConvert):
    """
    Convert to statsbomb coords
    1. Need to convert from meters to yards - done
    2. Flip and resize such that HawkEye coord system goes to statsbomb coord. system
    3. Flip x axis if needed - dependending on GK location - done
    """
    x, y = coords
    x = x * (60 / 52.5) + 60#step 1
    y = y * (40 / 43) + 40#not perfect but this is probably the closest we will get - make sure to change this in og buli code too
    #convert to statsbomb coord system here...
    y = 80 - y#step 2
    if needConvert:
        x = 120 - x
    return [x, y]

In [21]:
def read_player_loc(file_path, period, minute, second, team,actor, action_id, max_second = None):
    player_df_all = pd.read_json(file_path, lines = True, orient = 'columns')
    player_dict = player_df_all['samples'].loc[0]['people']
    player_df = pd.DataFrame(player_dict)#['centroid'].loc[0][0]
    player_df['time'] = player_df.apply(lambda d: d['centroid'][0]['time'], axis = 1)
    player_df = player_df.sort_values(by = ['time'])
    if not max_second:
        max_second = second + 1
    player_df = player_df[(player_df['time'] > second) & (player_df['time'] < max_second)]
    player_df['period'] = period
    player_df['minute'] = minute
    player_df['pos'] = player_df.apply(lambda d: d['centroid'][0]['pos'], axis = 1)
    player_df['statsbombid'] = player_df.apply(lambda d: d['personId']['uefaId'], axis = 1).astype(int).map(uefa_map)
    player_df = player_df.dropna()
    player_df['team'] = player_df['statsbombid'].astype(int).map(player_to_team)
    player_df['teammate'] = np.where(player_df['team'] == team, True, False) 
    player_df['isGK'] = np.where(player_df['statsbombid'].isin(goalkeepers), True, False)
    player_df['isActor'] = np.where(player_df['statsbombid'].astype(int) == actor, True, False)
    if player_df[(player_df['isGK']) & (player_df['teammate'])].reset_index(drop = True).loc[0]['pos'][0] > 0:#gotta be a better way to do this...
        player_df['NeedsFlip'] = True
    else:
        player_df['NeedsFlip'] = False
    #convert pos here
    player_df['conv_pos'] = player_df.apply(lambda row: convert_pos(row['pos'], row['NeedsFlip']), axis=1)
    player_df['freeze_frame'] = player_df.apply(lambda d: {'teammate':d['teammate'], 'keeper':d['isGK'], 'actor':d['isActor'], 
    'location':d['conv_pos']}, axis = 1)
    player_df['frameNum'] = player_df['time'].ne(player_df['time'].shift()).cumsum()
    player_df["event_uuid"]  = action_id + "-" + player_df['frameNum'].astype(str)
    player_df["visible_area"] = [[0,0, 120, 0, 120, 80, 0, 80, 0,0]] * player_df.shape[0]
    return player_df[['event_uuid', 'visible_area', 'freeze_frame']].reset_index(drop = True)
    #return player_df#.dropna()

In [22]:
def convert_to_360(df, idx):
    row = df.loc[idx]
    action_id = row['id']
    team = row['team']['id']
    period = row['period']
    time = row['timestamp']
    actor = row['player']['id']
    time_split = time.split(":")
    minute = int(time_split[-2]) + 1
    second = float(time_split[-1])

    player_loc_path = f"{player_tracking_dir}{file_path_begin}_{str(period)}_{str(minute)}.football.samples.centroids"
    #print(player_loc_path)
    player_df = read_player_loc(player_loc_path, period, minute, second, team, actor, action_id)
    all_dfs = [player_df]
    if(second > 59):
        print("Goes to next minute...")
        player_loc_path = f"{player_tracking_dir}{file_path_begin}_{str(period)}_{str(minute + 1)}.football.samples.centroids"
        player_df = read_player_loc(player_loc_path, period, minute, 0, team,actor, action_id, 60 - second)
        all_dfs.append(player_df)
    all_player_locs = pd.concat(all_dfs)
    return all_player_locs

In [23]:
#Next TODO: load event data
"""
General thought process:
1. Get number of frames(number of "passes" to add for each event) - call this number n
2. Right below the pass receipt, add n identical passes with the same values except the id will be og_id-k where k is the frame number
Will time be a problem? maybe problems with time
can use .loc[n.1] and then reset_index() maybe?
maybe use timestamp? concat and then sort by timestamp and period
"""
#events_df
def concat_event_data(num_frames, idx, event_df):
    row = event_df.loc[idx]
    action_id = row['id']
    df_to_add = pd.DataFrame([row] * num_frames)
    df_to_add['id'] = [f"{action_id}-{frame}" for frame in range(1,num_frames+1)]
    #print(df_to_add.columns)
    new_df = pd.concat([event_df, df_to_add]).sort_values(by = ['period', 'timestamp'])#.reset_index()
    return new_df

In [24]:
def save_to_json(path, df):
    json_str = df.to_json(orient='records')
    json_data = json.loads(json_str)
    #cleaned_data = remove_nan(json_data)

    # Save to a file
    with open(path, 'w') as json_file:
        json.dump(json_str, json_file, indent=2)

In [25]:
def get_frames(match, output_dir):
    game_id = matches_map[match]
    events_dir = f'/home/lz80/rdf/sp161/shared/soccer-decision-making/womens_euro_receipts/events/{game_id}.json'
    three_sixty_dir = f'/home/lz80/rdf/sp161/shared/soccer-decision-making/womens_euro_receipts/three-sixty/{game_id}.json'
    ball_dir = f"/home/lz80/rdf/sp161/shared/soccer-decision-making/HawkEye/HawkeyeUnzipped/{match}/{match}ball_loc.json"
    lineups = f"/home/lz80/rdf/sp161/shared/soccer-decision-making/womens_euro_receipts/lineups/{game_id}.json"
    #player_tracking_dir = f"/home/lz80/rdf/sp161/shared/soccer-decision-making/HawkEye/HawkeyeUnzipped/{sample_match}/{sample_match}player_loc.json"
    global player_tracking_dir
    global player_to_team
    global file_path_begin
    global goalkeepers
    global ball_locs
    player_tracking_dir = f"/home/lz80/rdf/sp161/shared/soccer-decision-making/HawkEye/HawkeyeUnzipped/{match}/scrubbed.samples.centroids/"
    event_output_dir = f"{output_dir}/events_added/{game_id}.json"
    three_sixty_output = f"{output_dir}/three-sixty_added/{game_id}.json"
    events_df = pd.read_json(events_dir, convert_dates = False)
    three_sixty_df = pd.read_json(three_sixty_dir, convert_dates = False)
    ball_locs = pd.read_json(ball_dir, convert_dates = False)
    lineup_df = pd.read_json(lineups, convert_dates = False)
    dirfiles = [f for f in listdir(player_tracking_dir) if not f.startswith('.')]
    sample_file = dirfiles[0]
    file_path_begin = "_".join(sample_file.split('_')[0:3])
    #player_locs = pd.read_json(player_tracking_dir, convert_dates = False)
    #player_locs['added_time'] = player_locs['added_time'].fillna(0)
    ball_locs['added_time'] = ball_locs['added_time'].fillna(0)
    team_1 = lineup_df['team_id'].loc[0]
    team_2 = lineup_df['team_id'].loc[1]
    team_1_dict = lineup_df['lineup'].loc[0]
    team_2_dict = lineup_df['lineup'].loc[1]
    team_1_lineup = [player_dict['player_id'] for player_dict in team_1_dict]
    team_2_lineup = [player_dict['player_id'] for player_dict in team_2_dict]
    team_map = {team_1 : team_1_lineup, team_2 : team_2_lineup}
    player_to_team = {player_id: team_id for team_id, players in team_map.items() for player_id in players}
    pos_dict = {player['player_id']: player['positions'][0]['position'] for player in team_1_dict if len(player['positions']) > 0}
    team_2_pos_dict = {player['player_id']: player['positions'][0]['position'] for player in team_2_dict if len(player['positions']) > 0}
    pos_dict.update(team_2_pos_dict)
    goalkeepers = [key for (key,value) in pos_dict.items() if value == "Goalkeeper"]
    needs_converting = events_df[events_df['pass'].notna()].index
    counter = 1
    for idx in needs_converting:
        print(f"\r{counter}/{needs_converting.shape[0]}",end = "", flush=True)
        counter = counter + 1
        new_360 = convert_to_360(events_df, idx)
        events_df = concat_event_data(new_360.shape[0], idx, events_df)
        three_sixty_df = pd.concat([three_sixty_df, new_360])
    save_to_json(events_df, event_output_dir)
    save_to_json(three_sixty_df, three_sixty_output)
    
    
    


In [ ]:
output_dir = "/home/lz80/rdf/sp161/shared/soccer-decision-making/hawkeye_one_game"
sample_match = '2032213_Germany_Denmark'
uefa_map = pd.read_csv("/home/lz80/un-xPass/unxpass/steffen/ID_matched.csv").set_index('ID').to_dict()['player_id']
get_frames(sample_match, output_dir)

13/774

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f52499cb640>>
Traceback (most recent call last):
  File "/home/lz80/un-xPass/.venv/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


24/774